In [2365]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup

Importing and combining 3 datasets from 3 different sites scraped by 3 different people

In [2366]:
jc = pd.read_csv('jobscentral.csv')
js = pd.read_csv('Jobstreet_Ziig.csv')
ft = pd.read_csv('salary_df_car_fut.csv')

In [2367]:
jc.head(1)

,Job Nature,Position Level,Job Category,Qualification,Salary,Company,Title,Description
0,Permanent,"Entry Level, Experienced","\r\n\r\nIT-Administration, \r\n \r\nIT-Soft...","Diploma, Bachelor's / Honours","\r\n\r\n 1,800 SGD - 2,000 SGD / Month\...",CareerBuilder Singapore,Junior Web Developer,[' • \xa0Provide first level of telephone and ...


In [2368]:
js.head(1)

,Unnamed: 0,Company,Title,Location,Address,Industry,Job Description,Seniority,Salary,Requirements,JD
0,0,Career Edge Asia Pte Ltd,Senior Data Analyst ($7K - $8K / CBD/ MNC ) re...,Singapore - Across Singapore,None,Human Resources Management/Consulting,"<div class=""unselectable wrap-text"" id=""job_de...",Min 3 years (Senior Executive),6000-8000,Degree in IT/ System engineering / Business /...,Responsibilities Responsible for dashboards an...


In [2369]:
ft.head(1)

,Unnamed: 0,Company,Title,Address,Emp_type,Seniority,Industry,Salary,Responsibility,Requirements
0,0,SKILLSFUTURE SINGAPORE AGENCY,"Executive, (Quality Management Division) (6-mo...","ONE MARINA BOULEVARD, 1 MARINA BOULEVARD 018989",Contract,NONE,Public / Civil Service,NONE,Roles & ResponsibilitiesResponsibilities\r\n\r...,NONE


In [2370]:
js.columns

Index(['Unnamed: 0', 'Company', 'Title', 'Location', 'Address', 'Industry',
       'Job Description', 'Seniority', 'Salary', 'Requirements', 'JD'],
      dtype='object')

In [2371]:
ft.columns

Index(['Unnamed: 0', 'Company', 'Title', 'Address', 'Emp_type', 'Seniority',
       'Industry', 'Salary', 'Responsibility', 'Requirements'],
      dtype='object')

In [2372]:
jc.columns

Index(['Job Nature', 'Position Level', 'Job Category', 'Qualification',
       'Salary', 'Company', 'Title', 'Description'],
      dtype='object')

In [2373]:
js.drop('Unnamed: 0', axis=1, inplace=True)
ft.drop('Unnamed: 0', axis=1, inplace=True)

In [2374]:
jc['Emp_type'] = jc['Job Nature']
jc.drop('Job Nature', axis=1, inplace=True)

In [2375]:
jc['Seniority'] = jc['Position Level']
jc.drop('Position Level', axis=1, inplace=True)

In [2376]:
js.drop('Job Description', axis=1, inplace=True)

In [2377]:
js['Description'] = js['JD']
js.drop('JD', axis=1, inplace=True)

In [2378]:
js['Description'] = js.Requirements + js.Description
js.drop('Requirements', axis=1, inplace=True)

In [2379]:
ft['Description'] = ft.Requirements + ft.Responsibility
ft.drop(['Requirements', 'Responsibility'], axis=1, inplace=True)

In [2380]:
ft.columns

Index(['Company', 'Title', 'Address', 'Emp_type', 'Seniority', 'Industry',
       'Salary', 'Description'],
      dtype='object')

Combined the 3 datasets into 1, proceeding to do EDA and cleaning

In [2381]:
df = pd.concat([js, jc, ft])

In [2382]:
df.reset_index(drop=True, inplace=True)

In [2383]:
df.shape

(4945, 11)

In [2384]:
df = df[['Title', 'Company', 'Job Category', 'Seniority', 'Industry', 'Emp_type', 'Location', 'Address', 'Description', 'Qualification', 'Salary']]

In [2385]:
def letterizer(input):
    return ''.join(char for char in input if char in 'abcedfghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')

Cleaning the job title, then converting to dummy variables

In [2386]:
df['Title'] = df.Title.apply(letterizer)
df['Title'] = ['Data Analyst' if 'data' in title.lower() and 'analyst' in title.lower() else title for title in df.Title]
df['Title'] = ['Data Scientist' if 'data' in title.lower() and 'scientist' in title.lower() else title for title in df.Title]
df['Title'] = ['Business Analyst' if 'business' in title.lower() and 'analyst' in title.lower() else title for title in df.Title]
df['Title'] = ['Data Engineer' if 'data' in title.lower() and 'engineer' in title.lower() else title for title in df.Title]
df['Title'] = ['Reseach Scientist' if 'research' in title.lower() and 'scientist' in title.lower() else title for title in df.Title]
df['Title'] = ['Data(others)' if 'data' in title.lower() and 'analyst' not in title.lower() and 'scientist' not in title.lower() and 'engineer' not in title.lower() else title for title in df.Title]
df['Title'] = ['Business(others)' if 'business' in title.lower() and 'analyst' not in title.lower() else title for title in df.Title]
df['Title'] = ['Analyst(others)' if 'analyst' in title.lower() and 'data' not in title.lower() and 'business' not in title.lower() else title for title in df.Title]
df['Title'] = ['Others' if title not in ['Data Analyst', 'Data Scientist', 
                                         'Business Analyst', 'Data Engineer', 'Reseach Scientist',
                                         'Data(others)', 'Business(others)', 'Analyst(others)'] else title for title in df.Title]

In [2387]:
for title in df.Title.unique():
    if title != 'Others':
        df['title_'+title] = [1 if title == row else 0 for row in df.Title]

Cleaning the company names

In [2388]:
df['Company'] = df.Company.apply(letterizer)

Cleaning the job categories - too many null values, so to preserve any possible value all non-null values will be thrown into the job description string before dropping the whole column

In [2389]:
print(df['Job Category'].isnull().sum())
df['Job Category'] = df['Job Category'].astype(str)
df['Job Category'] = df['Job Category'].apply(lambda x: x.replace('\r',''))
df['Job Category'] = df['Job Category'].apply(lambda x: x.replace('\n',''))

4200


In [2390]:
df['Description'] = [j if i == 'nan' else i + j for i, j in zip(df['Job Category'], df.Description)]
df.drop('Job Category', axis=1, inplace=True)

Cleaning location and address - dummy variables for location. Address will be left as a string.

In [2391]:
df['Location'] = [''.join(char for char in str(location) if char.lower() in ' abcedfghijklmnopqrstuvwxyz') for location in df.Location]

In [2392]:
df['Location'] = [location.replace('Singapore', '') for location in df.Location]
df['Location'] = ['Unknown' if Location.lower() == 'none' or Location.lower() == 'nan' or Location.lower() == ''else Location for Location in df.Location]
df['Location'] = ['Central' if 'central' in location.lower() else location for location in df.Location]
df['Location'] = ['Northeast' if 'northeast' in location.lower() else location for location in df.Location]
df['Location'] = ['Northwest' if 'northwest' in location.lower() else location for location in df.Location]
df['Location'] = ['East' if 'east' in location.lower() else location for location in df.Location]
df['Location'] = ['West' if 'west' in location.lower() else location for location in df.Location]
df['Location'] = ['North' if 'north' in location.lower() else location for location in df.Location]
df['Location'] = ['Across' if 'across' in location.lower() else location for location in df.Location]
df['Location'] = ['Overseas' if location not in df.Location.unique()[:6] else location for location in df.Location]

In [2393]:
df.Location.unique()

array(['Across', 'Central', 'West', 'Unknown', 'East', 'North',
       'Overseas'], dtype=object)

In [2394]:
for location in df.Location.unique():
    if location != 'Unknown':
        df['location_'+location] = [1 if location == row else 0 for row in df.Location]

In [2395]:
df['Address'] = [''.join(char for char in str(location) if char.lower() in ' abcedfghijklmnopqrstuvwxyz') for location in df.Address]
df['Address'] = ['Unknown' if address.lower() == 'none' or address.lower() == 'nan' or address.lower() == ''else address for address in df.Address]

Cleaning the qualifications - too messy and too many null fields. Chunk into the job description string and drop.

In [2396]:
print(df.Qualification.isnull().sum())
df['Description'] = [j if type(i) is float else i + j for i, j in zip(df.Qualification, df.Description)]
df.drop('Qualification', axis=1, inplace=True)

4200


Cleaning the salary data

In [2397]:
# temporarily putting unknown salary as 0
df['Salary'] = ['0' if salary == '0.0' or str(salary).lower() == 'none' else salary for salary in df.Salary]

In [2398]:
# temporarily putting below expectations as -1 and above expectations as 1 for future imputation
# expected salary is 4000

df['Salary'] = df.Salary.apply(lambda x: -1 if 'Below' in str(x) else x)
df['Salary'] = df.Salary.apply(lambda x: 4000 if 'Around' in str(x) else x)
df['Salary'] = df.Salary.apply(lambda x: 1 if 'Above' in str(x) else x)
df['Salary'] = df.Salary.apply(lambda x: ''.join([char for char in str(x) if char in '0123456789 - to']))

In [2399]:
def split1(value):
    splitted = value.split('-')
    if len(splitted) == 2:
        try:
            return str((int(splitted[0]) + int(splitted[1]))/2)
        except:
            return value
    else:
        return value

In [2400]:
def split2(value):
    splitted = value.split('to')
    if len(splitted) == 2:
        try:
            return str((int(splitted[0]) + int(splitted[1]))/2)
        except:
            return value
    else:
        return value

In [2401]:
df['Salary'] = df.Salary.apply(split1)
df['Salary'] = df.Salary.apply(split2)

In [2402]:
df['Salary'] = df.Salary.apply(lambda x: ''.join([char for char in x if char in '0123456789-.']))
df['Salary'] = df.Salary.apply(split1)
df['Salary'] = df.Salary.apply(float)

In [2403]:
df.Salary.describe()

count      4945.000000
mean       4466.744085
std       11496.020279
min          -1.000000
25%           1.000000
50%        3750.000000
75%        5500.000000
max      300000.000000
Name: Salary, dtype: float64

In [2404]:
# some seem to be yearly salary

df[df.Salary>15000].head()

,Title,Company,Seniority,Industry,Emp_type,Location,Address,Description,Salary,title_Data Analyst,...,title_Reseach Scientist,title_Data(others),title_Business(others),title_Data Engineer,location_Across,location_Central,location_West,location_East,location_North,location_Overseas
96,Data Analyst,Morgan McKinley,Min 2 years (Non-Executive),Human Resources Management/Consulting,NaN,Across,Raffles Place One Raffles Place Tower Singap...,NaN,57500.0,1,...,0,0,0,0,1,0,0,0,0,0
203,Data Analyst,Morgan McKinley,Min 5 years (Non-Executive),Human Resources Management/Consulting,NaN,Across,Raffles Place One Raffles Place Tower Singap...,Min 7-10 years of experience as Business Anal...,135000.0,1,...,0,0,0,0,1,0,0,0,0,0
219,Data Analyst,Morgan McKinley,Min 5 years (Non-Executive),Human Resources Management/Consulting,NaN,Across,Raffles Place One Raffles Place Tower Singap...,Min 7-10 years of experience as Business Anal...,135000.0,1,...,0,0,0,0,1,0,0,0,0,0
363,Analyst(others),Cobalt Consulting Asia Pte Ltd,Min 4 years (Non-Executive),Human Resources Management/Consulting,NaN,Across,Market Street Grace Global Raffles Singapore,NaN,66000.0,0,...,0,0,0,0,1,0,0,0,0,0
886,Data Scientist,Macdonald And Company Pte Ltd,Min 4 years (Manager),Human Resources Management/Consulting,NaN,Across,Unknown,Experience creating end to end data science s...,20000.0,0,...,0,0,0,0,1,0,0,0,0,0


In [2405]:
df['Salary'] = df.Salary.apply(lambda x: x/12 if x > 20000 else x)

In [2406]:
df[df.Salary > 20000]

# double-checked the posting to verify... guess the numbers are correct.

# AMAZON WEB SERVICES SINGAPORE PRIVATE LIMITEDSr. Data Architect, Data Warehousing & MPP

# AIA TOWER, 1 ROBINSON ROAD 048542

# Permanent

# Professional

# Information Technology
# $250,000to$300,000

# Annually

,Title,Company,Seniority,Industry,Emp_type,Location,Address,Description,Salary,title_Data Analyst,...,title_Reseach Scientist,title_Data(others),title_Business(others),title_Data Engineer,location_Across,location_Central,location_West,location_East,location_North,location_Overseas
3580,Data(others),DATASPARK PTE LTD,Middle Management,Information Technology,Permanent,Unknown,COMCENTRE EXETER ROAD,RequirementsQualifications 7+ years’ experien...,25000.000000,0,...,0,1,0,0,0,0,0,0,0,0
3596,Data(others),AMAZON WEB SERVICES SINGAPORE PRIVATE LIMITED,Professional,Information Technology,Permanent,Unknown,AIA TOWER ROBINSON ROAD,RequirementsBasic Qualifications BA/BS degree...,22916.666667,0,...,0,1,0,0,0,0,0,0,0,0


In [2407]:
# mean salary for valid numbers below 4000
below = df[(df.Salary != 0) & (df.Salary != 1) & (df.Salary != -1)].Salary[df[(df.Salary != 0) & (df.Salary != 1) & (df.Salary != -1)].Salary < 4000].mean()
print(below)
df['Salary'] = [below if salary == -1 else salary for salary in df.Salary]

3018.930936819172


In [2408]:
# mean salary for valid numbers above 4000
above = df[(df.Salary != 0) & (df.Salary != 1) & (df.Salary != -1)].Salary[df[(df.Salary != 0) & (df.Salary != 1) & (df.Salary != -1)].Salary > 4000].mean()
print(above)
df['Salary'] = [above if salary == 1 else salary for salary in df.Salary]

6906.916326877339


In [2409]:
# unknown salary values are still registered as 0.0 - impute or drop later.
df.Salary.value_counts().head()

0.000000       1046
6906.916327     653
4000.000000     410
6000.000000     147
5000.000000     128
Name: Salary, dtype: int64

Cleaning the job descriptions

In [2410]:
df['Description'] = df.Description.apply(lambda x: str(x).replace('\r', ''))
df['Description'] = df.Description.apply(lambda x: str(x).replace('\n', ''))

In [2411]:
# a lot of empty job descriptions... this isn't something that can be imputed. drop or proceed regardless?
df.Description.value_counts().head(1)

nan    1866
Name: Description, dtype: int64

Cleaning the seniority columns, and extracting the years from it to create a new column for minimum years of experience, then turning it into dummy variables

In [2412]:
df['Min Years'] = [int(''.join([char for char in seniority if char in '0123456789'])) if bool(re.search(r'\d', seniority)) is True else np.NaN for seniority in df.Seniority]

In [2413]:
# for null values, imputing the average number of years for a given job title
impute_dict = dict(zip(df.groupby('Title')['Min Years'].mean().index, df.groupby('Title')['Min Years'].mean()))
df['Min Years'] = [impute_dict[title] if year != year else year for year, title in zip(df['Min Years'], df['Title'])]

In [2414]:
df['Seniority'] = [''.join(char for char in seniority if char.lower() in 'abcedfghijklmnopqrstuvwxyz ') for seniority in df.Seniority]
df['Seniority'] = [seniority.replace('Min', '') for seniority in df.Seniority]
df['Seniority'] = [seniority.replace('  years ', '') for seniority in df.Seniority]
df['Seniority'] = [seniority.replace('  year ', '') for seniority in df.Seniority]
df['Seniority'] = ['Entry Level' if 'entry' in seniority.lower() and 'level' in seniority.lower() else seniority for seniority in df.Seniority]
df['Seniority'] = ['Unknown' if seniority.lower() == 'none' else seniority for seniority in df.Seniority]

In [2415]:
df['Seniority'] = ['Professional' if 'professional' in seniority.lower() else seniority for seniority in df.Seniority]
df['Seniority'] = ['Senior Executive' if 'Senior Executive' in seniority else seniority for seniority in df.Seniority]
df['Seniority'] = ['Senior Management' if 'Senior Management' in seniority else seniority for seniority in df.Seniority]
df['Seniority'] = ['Junior Executive' if 'Junior Executive' in seniority else seniority for seniority in df.Seniority]
df['Seniority'] = ['Manager' if 'Middle Management' in seniority else seniority for seniority in df.Seniority]
df['Seniority'] = ['Senior Manager' if 'Senior Manag' in seniority or 'Experienced Man' in seniority else seniority for seniority in df.Seniority]

In [2416]:
for seniority in df.Seniority.value_counts().head(7).index:
    df['seniority_'+seniority] = [1 if seniority == row else 0 for row in df.Seniority]

Cleaning the employment type column

In [2417]:
df['Emp_type'] = ['Unknown' if Emp_type != Emp_type or Emp_type == '0.0' or Emp_type.lower() == 'none'  else Emp_type for Emp_type in df['Emp_type']]

In [2418]:
# seems too varied, with too many null fields - throw into the description column and drop the rest
df.Emp_type.value_counts()

Unknown                                                              2855
Permanent                                                             743
Full Time                                                             503
Permanent, Full Time                                                  260
Contract                                                              241
Contract, Full Time                                                   143
Permanent, Contract                                                    94
Contract, Temporary                                                    39
Permanent, Contract, Full Time                                         16
Temporary                                                               7
Internship                                                              7
Part Time, Full Time, Internship                                        5
Permanent, Part Time                                                    5
Part Time                             

In [2419]:
df['Description'] = [j if i == 'Unknown' else i + j for i, j in zip(df.Emp_type, df.Description)]
df.drop('Emp_type', axis=1, inplace=True)

In [2420]:
df['Description'] = ['Unknown' if Description.lower() == 'none' or Description.lower() == 'nan' or Description.lower() == 'nonenone' or Description.lower() == '' else Description for Description in df.Description]

Cleaning and dummy-ing the Indstury column

In [2421]:
df['Industry'] = ['Unknown' if industry != industry or industry.lower() == 'none' else industry for industry in df['Industry']]
df['Industry'] = ['Banking and Finance' if 'banking' in industry.lower() or 'finance' in industry.lower() else industry for industry in df.Industry]
df['Industry'] = ['Information Technology' if 'information' in industry.lower() else industry for industry in df.Industry]
df['Industry'] = ['Human Resources Management/Consulting' if 'consult' in industry.lower() or 'human' in industry.lower() else industry for industry in df.Industry]
df['Industry'] = ['R&D' if 'r&d' in industry.lower() else industry for industry in df.Industry]


In [2422]:
df['Industry'] = ['Human Resources Management/Consulting' if 'consult' in industry.lower() or 'human' in industry.lower() else industry for industry in df.Industry]


In [2423]:
for industry in df.Industry.value_counts().head().index:
    if industry != 'Unknown':
        df['industry_'+industry] = [1 if row == industry else 0 for row in df.Industry]

Cleaning the company column

In [2424]:
df['Company'] = ['Unknown' if Company.lower() == 'none' or Company.lower() == 'nan' or Company.lower() == '' else Company for Company in df.Company]

Dropping columns that got turned into dummy variables

In [2425]:
df.drop('Title', axis=1, inplace=True)
df.drop('Location', axis=1, inplace=True)
df.drop('Seniority', axis=1, inplace=True)
df.drop('Industry', axis=1, inplace=True)

In [2426]:
df.dtypes

Company                                            object
Address                                            object
Description                                        object
Salary                                            float64
title_Data Analyst                                  int64
title_Analyst(others)                               int64
title_Business Analyst                              int64
title_Data Scientist                                int64
title_Reseach Scientist                             int64
title_Data(others)                                  int64
title_Business(others)                              int64
title_Data Engineer                                 int64
location_Across                                     int64
location_Central                                    int64
location_West                                       int64
location_East                                       int64
location_North                                      int64
location_Overs

In [2427]:
df.drop_duplicates(inplace=True)

In [2428]:
df.to_csv('final_df.csv', index=False)

In [2429]:
pd.read_csv('final_df.csv')

,Company,Address,Description,Salary,title_Data Analyst,title_Analyst(others),title_Business Analyst,title_Data Scientist,title_Reseach Scientist,title_Data(others),...,seniority_Junior Executive,seniority_Manager,seniority_Entry Level,seniority_Professional,seniority_Executive,seniority_Senior Manager,industry_Human Resources Management/Consulting,industry_Information Technology,industry_Banking and Finance,industry_R&D
0,Career Edge Asia Pte Ltd,Unknown,Degree in IT/ System engineering / Business /...,7000.000000,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,YGP Pte Ltd,Anson Road Singapore,Minimum 2 years of relevant experiences in Bu...,3500.000000,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,Unknown,Bukit Merah Central Podium,Unknown,4200.000000,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,nSearch Global Pte Ltd,Unknown,"Degree in Computer/Computer Science, Electroni...",6906.916327,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Adecco Personnel Pte Ltd,Unknown,Unknown,4000.000000,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
5,Recruitment Firm,Marina View Asia Square Tower Singapore,Unknown,3250.000000,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
6,RECRUIT EXPRESS PTE LTD,Unknown,Candidate must possess at least a Bachelor's ...,4000.000000,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,Sopra Steria Asia Pte Ltd,Fusionopolis Way Symbiosis,"Develop data mining, data analysis and data pr...",6906.916327,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
8,United Overseas Bank Limited UOB,Unknown,Minimum Bachelor Degree or equivalent profess...,6906.916327,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,United Overseas Bank Limited UOB,Unknown,Unknown,6906.916327,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
